In [ ]:
import numpy as np 
import tensorflow as tf

#load data set
x_real = np.loadtxt("train_x.csv", delimiter=",") # load from text 
y_real = np.loadtxt("train_y.csv", delimiter=",") 
#x = x.reshape(-1, 64, 64) # reshape 
#y = y.reshape(-1, 1) 

In [3]:
x_real_1 = np.where(x_real > 252, 1, 0)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 24, 25, 27, 28, 30, 32, 35, 36, 40, 42, 45, 48, 49, 54, 56, 63, 64, 72, 81]
# lb = preprocessing.LabelBinarizer()
# lb.fit(classes)


# x_train, x_test, y_train, y_test = train_test_split(x_real_1, y_real, test_size=0.25, random_state=42)

#normalize value in x_train


In [ ]:
#keras
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D , AveragePooling2D
from keras import backend as K
from keras.layers.normalization import BatchNormalization

batch_size = 100
num_classes = 40
epochs = 20

# input image dimensions
img_rows, img_cols = 64, 64

x_train, x_test, y_train, y_test = train_test_split(x_real_1, y_real, test_size=0.25, random_state=42)
lb = preprocessing.LabelBinarizer()
lb.fit(classes)
y_train = lb.transform(y_train)
y_test = lb.transform(y_test)


def build_model(conv,conv2, dense, batchNorm, learning_rate, is_pool_max, optimizer, dropout):
    global x_train, x_test, y_train, y_test
    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    # y_train = keras.utils.to_categorical(y_train, num_classes)
    # y_test = keras.utils.to_categorical(y_test, num_classes)

    model = Sequential()

    model.add(Conv2D(conv[0][0], kernel_size=(conv[0][1], conv[0][1]),input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    for conv_filter, kernel_size in conv[1:]:
        if conv_filter == None:
            if is_pool_max:
                model.add(MaxPooling2D(pool_size=(2, 2)))
            else:
                model.add(AveragePooling2D(pool_size=(2, 2)))
        else:
            model.add(Conv2D(conv_filter, (kernel_size, kernel_size)))
            if batchNorm:
                model.add(BatchNormalization())
            model.add(Activation('relu'))
    

    for conv_filter, kernel_size in conv2:
        if conv_filter == None:
            if is_pool_max:
                model.add(MaxPooling2D(pool_size=(2, 2)))
            else:
                model.add(AveragePooling2D(pool_size=(2, 2)))
        else:
            model.add(Conv2D(conv_filter, (kernel_size, kernel_size)))
            if batchNorm:
                model.add(BatchNormalization())
            model.add(Activation('relu'))

    model.add(Dropout(0.5))
    model.add(Flatten())

    for num_neuron in dense:
        
        model.add(Dense(num_neuron))
        if batchNorm:
                model.add(BatchNormalization())
        model.add(Activation('relu'))
        if dropout:
            model.add(Dropout(0.5))
   

    model.add(Dense(num_classes, activation='softmax'))

    from keras import optimizers
    if optimizer == "sdg":
        optimizer = optimizers.SGD(lr=learning_rate)
    elif optimizer == "adam":
        optimizer = optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    else:
        optimizer = optimizers.Adadelta(lr=learning_rate, rho=0.95, epsilon=1e-08, decay=0.0)
    # model.compile(loss='mean_squared_error', optimizer=sgd)
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=optimizer,
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))

    score = model.evaluate(x_test, y_test, verbose=0)
    
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    return (model, score)


convs = [
    [(64,5),(64,5),(None, None)],
    [(32,3),(32,3),(32,3),(None, None)],
    [(32,5),(32,5),(32,5),(None, None)],
    [(64,5),(32,3),(None, None)]
]
models = []
for conv in convs:
    for conv2 in convs:
        for dense in [[256],[512,256],[1024,512,256]]:
            for batchNorm in (True,):
                for learning_rate in [0.05]:
                    for is_pool_max in (True, False):
                        for optimizer in ["ada"]:
                            for dropout in (True,):
                                print(len(models) ,conv, conv2, dense, batchNorm, learning_rate, is_pool_max, optimizer, dropout)
                                models.append(build_model(conv,conv2, dense, batchNorm, learning_rate, is_pool_max, optimizer, dropout))
                                print(models[-1][0])



0 [(32, 5), (32, 5), (None, None)] [(32, 5), (32, 5), (None, None)] [256] True 0.05 True ada True
x_train shape: (15000, 64, 64, 1)
15000 train samples
5000 test samples
Train on 15000 samples, validate on 5000 samples
Epoch 1/20
15000/15000 [==============================] - 16s - loss: 4.4714 - acc: 0.0271 - val_loss: 3.6956 - val_acc: 0.0472
Epoch 2/20
15000/15000 [==============================] - 13s - loss: 4.3570 - acc: 0.0327 - val_loss: 3.6821 - val_acc: 0.0472
Epoch 3/20
15000/15000 [==============================] - 13s - loss: 4.2452 - acc: 0.0356 - val_loss: 3.6908 - val_acc: 0.0472
Epoch 4/20
15000/15000 [==============================] - 13s - loss: 4.1643 - acc: 0.0417 - val_loss: 3.6698 - val_acc: 0.0480
Epoch 5/20
15000/15000 [==============================] - 13s - loss: 4.0844 - acc: 0.0473 - val_loss: 3.6140 - val_acc: 0.0494
Epoch 6/20
15000/15000 [==============================] - 13s - loss: 4.0213 - acc: 0.0533 - val_loss: 3.5914 - val_acc: 0.0656
Epoch 7/20
15

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")


# # load json and create model
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [ ]:
# #keras
# from __future__ import print_function
# import keras
# from keras.datasets import mnist
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten,Activation
# from keras.layers import Conv2D, MaxPooling2D
# from keras import backend as K

# batch_size = 128
# num_classes = 10
# epochs = 12

# # input image dimensions
# img_rows, img_cols = 28, 28

# # the data, shuffled and split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# if K.image_data_format() == 'channels_first':
#     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#     x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
# else:
#     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols, 1)

# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255
# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# # convert class vectors to binary class matrices
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3),input_shape=input_shape))
# model.add(Activation('relu'))

# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))

# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Dropout(0.25))

# model.add(Flatten())

# model.add(Dense(128, activation='relu'))

# model.add(Dropout(0.5))

# model.add(Dense(num_classes, activation='softmax'))

# model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(x_test, y_test))

# score = model.evaluate(x_test, y_test, verbose=0)

# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

In [134]:

# def cnn_model_fn(features, labels, mode):
#     """Model function for CNN."""
#     # Input Layer
#     # Reshape X to 4-D tensor: [batch_size, width, height, channels]
#     # MNIST images are 28x28 pixels, and have one color channel
#     input_layer = tf.reshape(features["x"], [-1, 64, 64, 1])
#     conv1 = tf.layers.conv2d(
#       inputs=input_layer,
#       filters=32,
#       kernel_size=[5, 5],
#       padding="same",
#       activation=tf.nn.relu)
# #     conv1 = tf.nn.lrn(conv1)
#     #conv1 = tf.layers.batch_normalization(conv1)
# #     conv3 = tf.layers.conv2d(
# #       inputs=conv1,
# #       filters=32,
# #       kernel_size=[3, 3],
# #       padding="same",
# #       activation=tf.nn.relu)
    
# #     conv5 = tf.layers.conv2d(
# #       inputs=conv3,
# #       filters=32,
# #       kernel_size=[3, 3],
# #       padding="same",
# #       activation=tf.nn.relu)
# #     #conv2 = tf.layers.batch_normalization(conv2)
    
#     pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
#     # pool1 = tf.nn.lrn(pool1)
# #     conv2 = tf.layers.conv2d(
# #       inputs=pool1,
# #       filters=32,
# #       kernel_size=[3, 3],
# #       padding="same",
# #       activation=tf.nn.relu)
    
# #     conv4 = tf.layers.conv2d(
# #       inputs=conv2,
# #       filters=32,
# #       kernel_size=[3, 3],
# #       padding="same",
# #       activation=tf.nn.relu)
    
# #     conv2 = tf.nn.lrn(conv2)
# #     conv2 = tf.layers.batch_normalization(conv2)
# #     pool2 = tf.layers.max_pooling2d(inputs=conv4, pool_size=[2, 2], strides=2)

# #     # pool2 = tf.nn.lrn(pool2)
# #     pool2_flat = tf.reshape(pool2, [-1, 16 * 16 * 32])
#     pool2_flat = tf.reshape(pool1, [-1, 16 * 16 * 32])

#     dense = tf.layers.dense(inputs=pool2_flat, units=512, activation=tf.nn.relu)
# #     dense=tf.layers.batch_normalization(dense)
# #     dense2=tf.layers.batch_normalization(dense2)
#     dropout = tf.layers.dropout(
#       inputs=dense, rate=0.5, training=mode == tf.estimator.ModeKeys.TRAIN)
#     logits = tf.layers.dense(inputs=dropout, units=40)

#     predictions = {
#       # Generate predictions (for PREDICT and EVAL mode)
#       "classes": tf.argmax(input=logits, axis=1),
#       # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
#       # `logging_hook`.
#       "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
#     }
#     if mode == tf.estimator.ModeKeys.PREDICT:
#         return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

#     # Calculate Loss (for both TRAIN and EVAL modes)
#     onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=40)
#     loss = tf.losses.softmax_cross_entropy(
#       onehot_labels=onehot_labels, logits=logits)

#     # Configure the Training Op (for TRAIN mode)
#     if mode == tf.estimator.ModeKeys.TRAIN:
#         optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
#         train_op = optimizer.minimize(
#             loss=loss,
#             global_step=tf.train.get_global_step())
#         return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

#     # Add evaluation metrics (for EVAL mode)
#     eval_metric_ops = {
#       "accuracy": tf.metrics.accuracy(
#           labels=labels, predictions=predictions["classes"])}
#     return tf.estimator.EstimatorSpec(
#       mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [136]:
# mnist = tf.contrib.learn.datasets.load_dataset("mnist")
# train_data = mnist.train.images  # Returns np.array
# train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
# eval_data = mnist.test.images  # Returns np.array
# eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

In [ ]:
# #Create estimator
# mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,model_dir="/tmp/comp551_cnn_model21")

# #Set up Logging Hook
# tensors_to_log = {"probabilities": "softmax_tensor"}
# logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)

# #Start training model
# x_train = np.float32(x_train)
# x_test = np.float32(x_test)
# train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":x_train},y=y_train,batch_size=100,num_epochs=20 ,shuffle=False)
# mnist_classifier.train(input_fn=train_input_fn,hooks=[logging_hook])

# #Start evaluating model 
# eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":x_test},y=y_test,num_epochs=1,shuffle=False)
# eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
# print(eval_results)
                     
    
    
# # #Create estimator
# # mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,model_dir="/tmp/mnist_convnet_model_library2")

# # #Set up Logging Hook
# # tensors_to_log = {"probabilities": "softmax_tensor"}
# # logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)

# # #Start training model
# # x_train = np.float32(x_train)
# # x_test = np.float32(x_test)
# # train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":train_data},y=train_labels,batch_size=100,num_epochs=None,shuffle=True)
# # mnist_classifier.train(input_fn=train_input_fn,steps=5000,hooks=[logging_hook])

# # #Start evaluating model 
# # eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":eval_data},y=eval_labels,num_epochs=1,shuffle=False)
# # eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
# # print(eval_results)
                                                   
                                                   

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_service': None, '_is_chief': True, '_master': '', '_save_summary_steps': 100, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_task_id': 0, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_tf_random_seed': None, '_model_dir': '/tmp/comp551_cnn_model21', '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_session_config': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000018A0C5C9400>}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/comp551_cnn_model21\model.ckpt.
INFO:tensorflow:probabilities = [[ 0.47745761  0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 1.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 

INFO:tensorflow:step = 1101, loss = 3.03564 (2.130 sec)
INFO:tensorflow:global_step/sec: 47.2165
INFO:tensorflow:probabilities = [[ 0.06402103  0.02151576  0.02648365 ...,  0.01902647  0.01883078
   0.01734174]
 [ 0.06827712  0.02196723  0.02663695 ...,  0.0173307   0.01729674
   0.01620397]
 [ 0.0728052   0.01966844  0.02419657 ...,  0.01746782  0.01844065
   0.01664336]
 ..., 
 [ 0.06712171  0.019659    0.02416909 ...,  0.01722831  0.01863587
   0.01806602]
 [ 0.0743617   0.0203521   0.02475672 ...,  0.01796765  0.01795454
   0.01767302]
 [ 0.07249109  0.02108875  0.02402369 ...,  0.01777014  0.0188836
   0.01826647]] (2.116 sec)
INFO:tensorflow:step = 1201, loss = 3.07258 (2.121 sec)
INFO:tensorflow:global_step/sec: 47.0216
INFO:tensorflow:probabilities = [[ 0.07758945  0.02060453  0.02471991 ...,  0.01578524  0.01779322
   0.01509018]
 [ 0.08708433  0.02060778  0.02404752 ...,  0.01622344  0.0160259
   0.01623137]
 [ 0.07026418  0.02069583  0.02530133 ...,  0.0166702   0.01868347
 

INFO:tensorflow:step = 2401, loss = 2.98147 (2.135 sec)
INFO:tensorflow:global_step/sec: 47.0491
INFO:tensorflow:probabilities = [[ 0.12341935  0.01736703  0.02467198 ...,  0.00865309  0.00977789
   0.00965824]
 [ 0.09721713  0.01923605  0.02584895 ...,  0.01195831  0.0128112
   0.01269303]
 [ 0.11718263  0.01581454  0.02537143 ...,  0.00860514  0.00891841
   0.00873777]
 ..., 
 [ 0.10226037  0.01808525  0.02907392 ...,  0.01010147  0.01011517
   0.00987459]
 [ 0.13696353  0.01573259  0.02189991 ...,  0.00783926  0.00818442
   0.00810928]
 [ 0.12048472  0.019569    0.02584162 ...,  0.01121512  0.00997164
   0.00889242]] (2.125 sec)
INFO:tensorflow:step = 2501, loss = 2.86583 (2.125 sec)
INFO:tensorflow:global_step/sec: 46.9041
INFO:tensorflow:probabilities = [[ 0.12381633  0.01807002  0.02361465 ...,  0.0086781   0.00940237
   0.00962677]
 [ 0.11828086  0.01927512  0.02534042 ...,  0.0094414   0.0094064
   0.00905336]
 [ 0.12527975  0.01427575  0.02483694 ...,  0.0070522   0.00781204
 

INFO:tensorflow:step = 3701, loss = 2.89427 (2.131 sec)
INFO:tensorflow:global_step/sec: 47.0589
INFO:tensorflow:probabilities = [[ 0.13083437  0.0138152   0.02668992 ...,  0.00403171  0.00480045
   0.0041224 ]
 [ 0.10328098  0.01515036  0.03178052 ...,  0.00749813  0.00777483
   0.00597317]
 [ 0.12721787  0.01895278  0.02661904 ...,  0.00680201  0.00585354
   0.0062241 ]
 ..., 
 [ 0.11398745  0.01740892  0.02904648 ...,  0.00754108  0.00639858
   0.00658236]
 [ 0.12094074  0.01415156  0.02532211 ...,  0.0044452   0.0045681
   0.0045921 ]
 [ 0.13827451  0.01393103  0.02266354 ...,  0.00429641  0.00520384
   0.00445874]] (2.125 sec)
INFO:tensorflow:step = 3801, loss = 2.83147 (2.121 sec)
INFO:tensorflow:global_step/sec: 47.2342
INFO:tensorflow:probabilities = [[ 0.12244514  0.01372642  0.02579606 ...,  0.00478993  0.00530409
   0.00502923]
 [ 0.12059607  0.01621734  0.02796556 ...,  0.00542089  0.0046808
   0.00471475]
 [ 0.12136681  0.01685067  0.02670169 ...,  0.00467865  0.00585629
 

INFO:tensorflow:step = 5001, loss = 2.76367 (2.144 sec)
INFO:tensorflow:global_step/sec: 47.1443
INFO:tensorflow:probabilities = [[ 0.12080517  0.02288588  0.0312534  ...,  0.00688879  0.00577046
   0.00569419]
 [ 0.0984166   0.02108641  0.03002162 ...,  0.0051834   0.00526889
   0.00493347]
 [ 0.11187255  0.01985197  0.02566057 ...,  0.00665032  0.00676169
   0.00720645]
 ..., 
 [ 0.13083781  0.01651848  0.02863008 ...,  0.00278619  0.00276768
   0.00310259]
 [ 0.11659571  0.01728387  0.03021734 ...,  0.00444644  0.00509352
   0.00453556]
 [ 0.11782349  0.01585707  0.03165065 ...,  0.0049048   0.00479812
   0.00508285]] (2.121 sec)
INFO:tensorflow:step = 5101, loss = 2.98077 (2.125 sec)
INFO:tensorflow:global_step/sec: 47.0646
INFO:tensorflow:probabilities = [[ 0.12119427  0.02148616  0.02521858 ...,  0.00364799  0.00406912
   0.00408834]
 [ 0.09760486  0.02201745  0.03071036 ...,  0.00710331  0.00697616
   0.00616479]
 [ 0.11493317  0.01597162  0.02977272 ...,  0.0041884   0.00519955